In [ ]:
import pandas as pd
import os

In [ ]:
#Import Raw Data into Python
os.chdir('/Users/maxwell/Desktop/Code/SALMONQUEST/SalmonData')

datapath = 'all_dams.csv'
rawdata = pd.DataFrame(pd.read_csv(datapath))

In [ ]:
pd.options.display.max_rows=40

In [ ]:
#This attribute provides the dataset's techincal info.
rawdata.info()

In [ ]:
#Let's examine the database
rawdata

In [ ]:
"""

I need two datasets. 

The first has two columns: date and total salmon counts. I'll call it data_simple
This answers the general question I asked at the outset of this project:
When should I fish to have the highest likelihood of catching a salmon?

The second includes salmon numbers for each of the three types at each of the 9 location centers.
I'll call this data_detailed
This dataset can be used to answer a more specific question:
Which section of the Columbia river should I fish on, and on which day of the year, 
to have the highest chance of catching each of the three salmon types?

"""

In [ ]:
#Reformat 1: Rename columns
d1 = rawdata.rename(
    columns={
        'Project': 'Location',
        'Chin': 'Chinook',
        'Sock': 'Sockeye',
    })

#Reformat 2: Remove all fish types that aren't adult salmon.
#Only Chinook, Sockeye, and Coho are the ones I would eat for dinner !
column_names = ['Location', 'Date', 'Chinook','Sockeye','Coho']
d2 = d1[column_names]

In [ ]:
#QUESTION:
#What do rows with null date values look like?
non_data = d2.loc[d2['Date'].isna(), ['Location', 'Sockeye']]
non_data.head(20)
#They are notes from the tables stored in the location column. These are not data points.

In [ ]:
#Reformat 3: Remove all null date values.
d3 = d2[d2['Date'].notna()]

#Reformat 4: Change null salmon counts to zeroes.
d4 = d3.fillna(0)

#Reformat 5: Order entries by Date; then by Location
d5 = d4.sort_values(by=['Date', 'Location'], ignore_index=True)

#Reformat 6: Add total salmon count column.
d5['Total Salmon'] = (d5['Chinook']+d5['Sockeye']+d5['Coho'])

In [ ]:
#Create the simple dataset.

# Include Date and Total Salmon columns only.
ds1 = d5[['Date', 'Total Salmon']]
#Take average of all salmon count data points for each day of the year.
ds2 = ds1.groupby('Date', as_index=False)['Total Salmon'].mean()
ds2['Total Salmon'] = ds2['Total Salmon'].astype(int)

#Check ds2 has an entry for every day of the year.
len(ds2) == 365

data_simple = ds2

In [ ]:
#Export the simple dataset.

data_simple.to_csv('data_simple.csv', index=False)

In [ ]:
#Create the detailed dataset.

#Create salmon run numbers of zero for each date that there is no entry at each location.